In [ ]:
import base64
from requests_toolbelt.multipart import decoder
import re
import boto3

In [ ]:
def handler(event, context):
    body = "..."
# body у каждой картинки свой
    decoded_body = base64.b64decode(body)
    content_type = "multipart/form-data; boundary=--------------------------388679767199243188882969"
# boundary у каждого изображения различается
    def get_field_name(content_disposition):
        parts = content_disposition.split(';')
        for part in parts:
            part_stripped = part.strip()
            search_result = re.search("^name=\"(.*)\"$", part_stripped)
        
            if search_result:
                return search_result.group(1)
                                
        for part in decoder.MultipartDecoder(decoded_body, content_type).parts: 
            content_disposition = part.headers[b'Content-Disposition'].decode('utf-8');
            print("content_disposition: " + content_disposition)
            field_name = get_field_name(content_disposition)
        
        session = boto3.session.Session()
        s3 = session.client(
                service_name='s3',
                endpoint_url='https://storage.yandexcloud.net'
        )       
        s3.put_object(Bucket='image-handler-baket', Key='sample', Body='TEST', StorageClass='STANDARD')
    